In [1551]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
plt.rcParams['figure.figsize'] = [15,8]
import warnings
warnings.filterwarnings('ignore')

from xverse.transformer import WOE
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report,f1_score,accuracy_score,cohen_kappa_score
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from scipy import stats


In [1552]:
df = pd.read_csv('Train.csv')
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome
0,526927,6570,14/05/05,0,0,0,0,0,0
1,510379,6534,26/05/06,0,0,0,0,0,1
2,520968,6557,07/01/04,0,0,0,0,0,1
3,507625,6535,12/02/04,0,0,0,0,0,0
4,502611,6581,14/03/04,0,0,0,0,0,0


In [1553]:
df.describe()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,outcome
count,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000
mean,507203.621722,6547.598246,0.840266,0.258492,0.000418,0.022545,0.253122,0.272137
std,12408.581003,19.265787,8.081947,3.982679,0.020429,0.200734,1.251085,0.445064
min,485679.000000,6523.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,496444.250000,6534.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,507241.000000,6541.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,517919.000000,6562.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,528657.000000,6587.000000,288.000000,156.000000,1.000000,4.000000,31.000000,1.000000


In [1554]:
pad = pd.read_csv('Patient_Details.csv')
pad.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


In [1555]:
hcd = pd.read_csv('Health_Camp_Detail.csv')
hcd.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [1556]:
h1 = pd.read_csv('First_Health_Camp.csv')
h1.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [1557]:
h1.drop('Unnamed: 4',1,inplace=True)
h1.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score
0,506181,6560,40,0.439024
1,494977,6560,20,0.097561
2,518680,6560,10,0.048780
3,509916,6560,30,0.634146
4,488006,6560,20,0.024390


In [1558]:
h2 = pd.read_csv('Second_Health_Camp.csv')
h2.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [1559]:
h3 = pd.read_csv('Third_Health_Camp.csv')
h3.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [1560]:
df1 = pd.read_csv('Test.csv')
df1.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,507518,6529,02/04/06,0,0,0,0,0
1,516180,6570,13/05/05,0,0,0,0,0
2,507204,6537,16/01/07,0,0,0,0,0
3,505815,6578,27/07/05,0,0,0,0,0
4,501809,6542,22/07/05,0,0,0,0,0


In [1561]:
hcd.describe(include='O')

,Camp_Start_Date,Camp_End_Date,Category1,Category2
count,65,65,65,65
unique,58,54,3,7
top,16-Aug-03,07-Nov-07,First,C
freq,2,5,44,16


In [1562]:
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome
0,526927,6570,14/05/05,0,0,0,0,0,0
1,510379,6534,26/05/06,0,0,0,0,0,1
2,520968,6557,07/01/04,0,0,0,0,0,1
3,507625,6535,12/02/04,0,0,0,0,0,0
4,502611,6581,14/03/04,0,0,0,0,0,0


In [1563]:
df = pd.merge(df,pad,on='Patient_ID')

In [1564]:
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN
1,526927,6539,23/11/04,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN
2,510379,6534,26/05/06,0,0,0,0,0,1,0,0,0,0,None,None,None,26-May-06,H,NaN
3,520968,6557,07/01/04,0,0,0,0,0,1,0,0,0,0,None,None,None,07-Jan-04,H,NaN
4,507625,6535,12/02/04,0,0,0,0,0,0,0,0,0,0,None,None,None,12-Feb-04,B,NaN


In [1565]:
df = pd.merge(df,hcd,on='Health_Camp_ID')
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2


In [1566]:
df = pd.merge(df,h1,on=['Patient_ID','Health_Camp_ID'],how='left')
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2,40.0,0.191336


In [1567]:
df = pd.merge(df,h2,on=['Patient_ID','Health_Camp_ID'],how='left')
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score,Health Score
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2,40.0,0.191336,NaN


In [1568]:
df = pd.merge(df,h3,on=['Patient_ID','Health_Camp_ID'],how='left')
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2,40.0,0.191336,NaN,NaN,NaN


In [1569]:
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2,40.0,0.191336,NaN,NaN,NaN


In [1570]:
df.isnull().sum()/len(df)*100

Patient_ID                    0.000000
Health_Camp_ID                0.000000
Registration_Date             0.438380
Var1                          0.000000
Var2                          0.000000
Var3                          0.000000
Var4                          0.000000
Var5                          0.000000
outcome                       0.000000
Online_Follower               0.000000
LinkedIn_Shared               0.000000
Twitter_Shared                0.000000
Facebook_Shared               0.000000
Income                        0.000000
Education_Score               0.000000
Age                           0.000000
First_Interaction             0.000000
City_Type                    44.096102
Employer_Category            79.885755
Camp_Start_Date               0.000000
Camp_End_Date                 0.000000
Category1                     0.000000
Category2                     0.000000
Category3                     0.000000
Donation                     91.731506
Health_Score             

In [1571]:
df[df['Health_Camp_ID'] == 6570]

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2,40.0,0.191336,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,523810,6570,06/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,01-May-05,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
2516,523835,6570,20/06/05,0,0,0,0,0,0,0,0,0,0,None,None,None,18-Jun-05,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
2517,503273,6570,12/07/05,0,0,0,0,0,0,0,0,0,0,None,None,None,12-Jul-05,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
2518,511827,6570,13/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,29-Oct-03,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN


In [1572]:
df.shape

(52694, 29)

In [1573]:
df[df['Category1'] == 'First']

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN,NaN,NaN
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2,40.0,0.191336,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52689,511005,6563,12/09/04,0,0,0,0,0,0,0,0,0,0,None,None,None,27-Aug-04,NaN,NaN,09-Sep-04,14-Sep-04,First,C,2,NaN,NaN,NaN,NaN,NaN
52690,514187,6563,11/09/04,0,0,0,0,0,1,0,0,0,0,None,None,None,08-Sep-04,NaN,NaN,09-Sep-04,14-Sep-04,First,C,2,30.0,0.752137,NaN,NaN,NaN
52691,487712,6563,11/09/04,0,0,0,0,0,0,0,0,0,0,None,None,None,02-Sep-04,NaN,NaN,09-Sep-04,14-Sep-04,First,C,2,NaN,NaN,NaN,NaN,NaN
52692,487214,6563,11/09/04,0,0,0,0,0,0,0,0,0,0,None,None,None,08-Sep-04,NaN,NaN,09-Sep-04,14-Sep-04,First,C,2,NaN,NaN,NaN,NaN,NaN


In [1574]:
df.drop('Donation',1,inplace=True)

In [1575]:
dum = df.copy()

In [1576]:
df['Health_Score'].fillna(df['Health Score']).isnull().sum()

42857

In [1577]:
df['Health_Score'].isnull().sum()

48337

In [1578]:
df[df['Health Score'].isnull() == False]['Health_Score'].isnull().value_counts()

True    5480
Name: Health_Score, dtype: int64

In [1579]:
df['Health_Score'] = df['Health_Score'].fillna(df['Health Score'])

In [1580]:
df.drop('Health Score',1,inplace=True)
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Health_Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,526927,6570,14/05/05,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
1,487380,6570,15/05/05,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
2,521555,6570,30/05/05,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
3,511127,6570,14/07/05,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,09-Jul-05,22-Jul-05,First,E,2,NaN,NaN,NaN
4,498702,6570,13/05/05,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,09-Jul-05,22-Jul-05,First,E,2,0.191336,NaN,NaN


In [1581]:
df.drop(['Number_of_stall_visited','Last_Stall_Visited_Number',
         'Camp_Start_Date','Camp_End_Date','Registration_Date'],1,inplace=True)

In [1582]:
df.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,526927,6570,0,0,0,0,0,0,0,0,0,0,None,None,None,14-Nov-04,NaN,NaN,First,E,2,NaN
1,487380,6570,0,0,0,0,0,0,0,0,0,0,3,69,53,21-Sep-04,B,Education,First,E,2,NaN
2,521555,6570,6,1,0,0,3,0,0,0,0,0,3,86,47,03-Jul-03,B,Software Industry,First,E,2,NaN
3,511127,6570,0,0,0,0,0,0,0,1,0,1,0,None,39,08-Feb-03,C,NaN,First,E,2,NaN
4,498702,6570,0,0,0,0,0,1,0,0,0,0,1,69,40,22-Dec-02,E,Software Industry,First,E,2,0.191336


In [1583]:
df.drop('First_Interaction',1,inplace=True)
df.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,526927,6570,0,0,0,0,0,0,0,0,0,0,None,None,None,NaN,NaN,First,E,2,NaN
1,487380,6570,0,0,0,0,0,0,0,0,0,0,3,69,53,B,Education,First,E,2,NaN
2,521555,6570,6,1,0,0,3,0,0,0,0,0,3,86,47,B,Software Industry,First,E,2,NaN
3,511127,6570,0,0,0,0,0,0,0,1,0,1,0,None,39,C,NaN,First,E,2,NaN
4,498702,6570,0,0,0,0,0,1,0,0,0,0,1,69,40,E,Software Industry,First,E,2,0.191336


In [1584]:
df['Health_Score'] = df['Health_Score'].fillna(df['Health_Score'].median())

In [1585]:
df.isnull().sum()

Patient_ID               0
Health_Camp_ID           0
Var1                     0
Var2                     0
Var3                     0
Var4                     0
Var5                     0
outcome                  0
Online_Follower          0
LinkedIn_Shared          0
Twitter_Shared           0
Facebook_Shared          0
Income                   0
Education_Score          0
Age                      0
City_Type            23236
Employer_Category    42095
Category1                0
Category2                0
Category3                0
Health_Score             0
dtype: int64

In [1586]:
df['City_Type'] = df['City_Type'].fillna(df['City_Type'].mode()[0])

In [1587]:
df['Employer_Category'].describe()

count          10599
unique            14
top       Technology
freq            2505
Name: Employer_Category, dtype: object

In [1588]:
df['Employer_Category'] = df['Employer_Category'].fillna(df['Employer_Category'].mode()[0])

In [1589]:
df.isnull().sum()

Patient_ID           0
Health_Camp_ID       0
Var1                 0
Var2                 0
Var3                 0
Var4                 0
Var5                 0
outcome              0
Online_Follower      0
LinkedIn_Shared      0
Twitter_Shared       0
Facebook_Shared      0
Income               0
Education_Score      0
Age                  0
City_Type            0
Employer_Category    0
Category1            0
Category2            0
Category3            0
Health_Score         0
dtype: int64

In [1590]:
df.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,526927,6570,0,0,0,0,0,0,0,0,0,0,None,None,None,B,Technology,First,E,2,0.519169
1,487380,6570,0,0,0,0,0,0,0,0,0,0,3,69,53,B,Education,First,E,2,0.519169
2,521555,6570,6,1,0,0,3,0,0,0,0,0,3,86,47,B,Software Industry,First,E,2,0.519169
3,511127,6570,0,0,0,0,0,0,0,1,0,1,0,None,39,C,Technology,First,E,2,0.519169
4,498702,6570,0,0,0,0,0,1,0,0,0,0,1,69,40,E,Software Industry,First,E,2,0.191336


In [1591]:
df['Var1'] = df['Var1'].astype('O')

In [1592]:
df['Var2'] = df['Var2'].astype('O')
df['Var3'] = df['Var3'].astype('O')
df['Var4'] = df['Var4'].astype('O')
df['Var5'] = df['Var5'].astype('O')

In [1593]:
df.describe()

,Patient_ID,Health_Camp_ID,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Category3,Health_Score
count,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000,52694.000000
mean,507203.621722,6547.598246,0.272137,0.055642,0.065207,0.052947,0.055168,1.994724,0.522621
std,12408.581003,19.265787,0.445064,0.229231,0.246893,0.223930,0.228309,0.072443,0.116587
min,485679.000000,6523.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.001667
25%,496444.250000,6534.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.519169
50%,507241.000000,6541.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.519169
75%,517919.000000,6562.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.519169
max,528657.000000,6587.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000


In [1594]:
df.describe(include='O')

,Var1,Var2,Var3,Var4,Var5,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2
count,52694,52694,52694,52694,52694,52694,52694,52694,52694,52694,52694,52694
unique,120,71,2,5,30,8,211,50,9,14,3,7
top,0,0,0,0,0,None,None,None,B,Technology,First,F
freq,47284,50989,52672,51891,48546,37496,45711,36108,29050,44600,34990,17316


In [1595]:
df['Age'] = df['Age'].replace('None',np.nan)

In [1596]:
df['Education_Score'] = df['Education_Score'].replace('None',np.nan)
df['Income'] = df['Income'].replace('None',np.nan)

In [1597]:
df.describe(include='O')

,Var1,Var2,Var3,Var4,Var5,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2
count,52694,52694,52694,52694,52694,15198,6983,16586,52694,52694,52694,52694
unique,120,71,2,5,30,7,210,49,9,14,3,7
top,0,0,0,0,0,0,82,41,B,Technology,First,F
freq,47284,50989,52672,51891,48546,5882,343,1273,29050,44600,34990,17316


In [1598]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Income'] = df['Income'].fillna(df['Income'].median())
df['Education_Score'] = df['Education_Score'].fillna(df['Education_Score'].median())

In [1599]:
df.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,526927,6570,0,0,0,0,0,0,0,0,0,0,1.0,79.0,44.0,B,Technology,First,E,2,0.519169
1,487380,6570,0,0,0,0,0,0,0,0,0,0,3,69,53,B,Education,First,E,2,0.519169
2,521555,6570,6,1,0,0,3,0,0,0,0,0,3,86,47,B,Software Industry,First,E,2,0.519169
3,511127,6570,0,0,0,0,0,0,0,1,0,1,0,79.0,39,C,Technology,First,E,2,0.519169
4,498702,6570,0,0,0,0,0,1,0,0,0,0,1,69,40,E,Software Industry,First,E,2,0.191336


In [1600]:
from sklearn.preprocessing import LabelEncoder
LE1 = LabelEncoder()
df['City_Type'] = LE1.fit_transform(df['City_Type'])

In [1601]:
LE2 = LabelEncoder()
df['Employer_Category'] = LE2.fit_transform(df['Employer_Category'])

In [1602]:
LE3 = LabelEncoder()
df['Category1'] = LE3.fit_transform(df['Category1'])

In [1603]:
LE4 = LabelEncoder()
df['Category2'] = LE4.fit_transform(df['Category2'])

In [1604]:
df.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,outcome,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,526927,6570,0,0,0,0,0,0,0,0,0,0,1.0,79.0,44.0,1,11,0,4,2,0.519169
1,487380,6570,0,0,0,0,0,0,0,0,0,0,3,69,53,1,3,0,4,2,0.519169
2,521555,6570,6,1,0,0,3,0,0,0,0,0,3,86,47,1,10,0,4,2,0.519169
3,511127,6570,0,0,0,0,0,0,0,1,0,1,0,79.0,39,2,11,0,4,2,0.519169
4,498702,6570,0,0,0,0,0,1,0,0,0,0,1,69,40,4,10,0,4,2,0.191336


In [1605]:
df.drop(['Patient_ID','Health_Camp_ID'],1,inplace=True)

In [1606]:
df.dtypes

Var1                  object
Var2                  object
Var3                  object
Var4                  object
Var5                  object
outcome                int64
Online_Follower        int64
LinkedIn_Shared        int64
Twitter_Shared         int64
Facebook_Shared        int64
Income                object
Education_Score       object
Age                   object
City_Type              int32
Employer_Category      int32
Category1              int32
Category2              int32
Category3              int64
Health_Score         float64
dtype: object

In [1607]:
df[['outcome','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared']] = df[['outcome','Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared']].astype('O')

In [1608]:
df.dtypes

Var1                  object
Var2                  object
Var3                  object
Var4                  object
Var5                  object
outcome               object
Online_Follower       object
LinkedIn_Shared       object
Twitter_Shared        object
Facebook_Shared       object
Income                object
Education_Score       object
Age                   object
City_Type              int32
Employer_Category      int32
Category1              int32
Category2              int32
Category3              int64
Health_Score         float64
dtype: object

In [1609]:
df[['Income','Education_Score','Age']] = df[['Income','Education_Score','Age']].astype(float)

In [1610]:
x = df.drop('outcome',1)
y = df['outcome']
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=10,stratify=y)

In [1611]:
xtrain.head()

,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
37786,0,0,0,0,0,0,0,0,0,0.0,79.0,69.0,3,11,1,0,2,0.803378
24881,0,0,0,0,0,0,0,0,0,1.0,79.0,44.0,0,11,0,5,2,0.288382
38164,0,0,0,0,0,0,0,0,0,1.0,79.0,44.0,5,11,1,0,2,0.519169
18675,0,0,0,0,0,0,0,0,0,1.0,79.0,44.0,2,11,0,5,2,0.360339
28016,0,0,0,0,0,0,0,0,0,1.0,79.0,44.0,2,11,2,6,2,0.519169


In [1612]:
SS = StandardScaler()
xtrain.iloc[:,[9,10,11,17]] = SS.fit_transform(xtrain.iloc[:,[9,10,11,17]])
xtest.iloc[:,[9,10,11,17]] = SS.transform(xtest.iloc[:,[9,10,11,17]])

In [1613]:
xtrain.head()

,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
37786,0,0,0,0,0,0,0,0,0,-1.416751,0.039517,3.506197,3,11,1,0,2,2.410116
24881,0,0,0,0,0,0,0,0,0,-0.111178,0.039517,-0.184911,0,11,0,5,2,-2.009278
38164,0,0,0,0,0,0,0,0,0,-0.111178,0.039517,-0.184911,5,11,1,0,2,-0.028794
18675,0,0,0,0,0,0,0,0,0,-0.111178,0.039517,-0.184911,2,11,0,5,2,-1.391787
28016,0,0,0,0,0,0,0,0,0,-0.111178,0.039517,-0.184911,2,11,2,6,2,-0.028794


In [1614]:
xtrain.dtypes

Var1                  object
Var2                  object
Var3                  object
Var4                  object
Var5                  object
Online_Follower       object
LinkedIn_Shared       object
Twitter_Shared        object
Facebook_Shared       object
Income               float64
Education_Score      float64
Age                  float64
City_Type              int32
Employer_Category      int32
Category1              int32
Category2              int32
Category3              int64
Health_Score         float64
dtype: object

In [1615]:
xtrain = xtrain.astype(np.number)

In [1616]:
ytrain = ytrain.astype(np.number)

In [1617]:
ytest = ytest.astype(np.number)

#### M0

In [1618]:
lr = LogisticRegression()
model = lr.fit(xtrain,ytrain)
ypred = model.predict(xtest)
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

         0.0       0.82      0.88      0.85     11507
         1.0       0.61      0.49      0.55      4302

    accuracy                           0.78     15809
   macro avg       0.72      0.69      0.70     15809
weighted avg       0.77      0.78      0.77     15809



In [1619]:
ytest.value_counts(normalize=True)*100

0.0    72.787653
1.0    27.212347
Name: outcome, dtype: float64

#### M1

In [1620]:
rf = RandomForestClassifier()
model = rf.fit(xtrain,ytrain)
ypred = model.predict(xtest)
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

         0.0       0.99      0.94      0.96     11507
         1.0       0.85      0.98      0.91      4302

    accuracy                           0.95     15809
   macro avg       0.92      0.96      0.94     15809
weighted avg       0.95      0.95      0.95     15809



In [1621]:
ypred = model.predict(xtrain)
print(classification_report(ytrain,ypred))

              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97     26847
         1.0       0.87      1.00      0.93     10038

    accuracy                           0.96     36885
   macro avg       0.93      0.97      0.95     36885
weighted avg       0.96      0.96      0.96     36885



In [1670]:
from sklearn.model_selection import KFold,cross_val_score

In [1671]:
x.iloc[:,[9,10,11,17]] = SS.transform(x.iloc[:,[9,10,11,17]])

In [1672]:
y = y.astype(np.number)

In [1676]:

c = cross_val_score(estimator=RandomForestClassifier(),X=x,y=y,cv=5,scoring='accuracy')

In [1679]:
np.mean(c)

0.9348128743618922

In [1622]:
ypred

array([1., 1., 0., ..., 1., 0., 0.])

In [1623]:
df1.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,507518,6529,02/04/06,0,0,0,0,0
1,516180,6570,13/05/05,0,0,0,0,0
2,507204,6537,16/01/07,0,0,0,0,0
3,505815,6578,27/07/05,0,0,0,0,0
4,501809,6542,22/07/05,0,0,0,0,0


In [1624]:
df1 = pd.merge(df1,pad,on='Patient_ID')

In [1625]:
df1 = pd.merge(df1,hcd,on='Health_Camp_ID')
df1.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,507518,6529,02/04/06,0,0,0,0,0,0,0,0,0,None,None,None,07-Jan-05,B,NaN,30-Mar-06,03-Apr-06,Second,A,2
1,511232,6529,03/03/06,0,0,0,0,0,0,0,0,0,None,None,None,29-Jun-04,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2
2,507817,6529,15/02/06,0,0,0,0,0,0,0,0,0,None,None,None,13-Jul-04,E,NaN,30-Mar-06,03-Apr-06,Second,A,2
3,500528,6529,27/03/06,0,0,0,0,0,0,0,0,0,None,None,None,27-Mar-06,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2
4,503972,6529,19/02/06,1,0,0,0,0,0,0,0,0,1,64,63,05-Dec-02,B,Others,30-Mar-06,03-Apr-06,Second,A,2


In [1626]:
df1 = pd.merge(df1,h1,on=['Patient_ID','Health_Camp_ID'],how='left')
df1.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score
0,507518,6529,02/04/06,0,0,0,0,0,0,0,0,0,None,None,None,07-Jan-05,B,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN
1,511232,6529,03/03/06,0,0,0,0,0,0,0,0,0,None,None,None,29-Jun-04,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN
2,507817,6529,15/02/06,0,0,0,0,0,0,0,0,0,None,None,None,13-Jul-04,E,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN
3,500528,6529,27/03/06,0,0,0,0,0,0,0,0,0,None,None,None,27-Mar-06,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN
4,503972,6529,19/02/06,1,0,0,0,0,0,0,0,0,1,64,63,05-Dec-02,B,Others,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN


In [1627]:
df1 = pd.merge(df1,h2,on=['Patient_ID','Health_Camp_ID'],how='left')
df1.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score,Health Score
0,507518,6529,02/04/06,0,0,0,0,0,0,0,0,0,None,None,None,07-Jan-05,B,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,0.065742
1,511232,6529,03/03/06,0,0,0,0,0,0,0,0,0,None,None,None,29-Jun-04,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,NaN
2,507817,6529,15/02/06,0,0,0,0,0,0,0,0,0,None,None,None,13-Jul-04,E,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,0.338359
3,500528,6529,27/03/06,0,0,0,0,0,0,0,0,0,None,None,None,27-Mar-06,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,NaN
4,503972,6529,19/02/06,1,0,0,0,0,0,0,0,0,1,64,63,05-Dec-02,B,Others,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,0.572376


In [1628]:
df1 = pd.merge(df1,h3,on=['Patient_ID','Health_Camp_ID'],how='left')
df1.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,507518,6529,02/04/06,0,0,0,0,0,0,0,0,0,None,None,None,07-Jan-05,B,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,0.065742,NaN,NaN
1,511232,6529,03/03/06,0,0,0,0,0,0,0,0,0,None,None,None,29-Jun-04,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,NaN,NaN,NaN
2,507817,6529,15/02/06,0,0,0,0,0,0,0,0,0,None,None,None,13-Jul-04,E,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,0.338359,NaN,NaN
3,500528,6529,27/03/06,0,0,0,0,0,0,0,0,0,None,None,None,27-Mar-06,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,NaN,NaN,NaN
4,503972,6529,19/02/06,1,0,0,0,0,0,0,0,0,1,64,63,05-Dec-02,B,Others,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,0.572376,NaN,NaN


In [1629]:
df1.isnull().sum()/len(df1)*100

Patient_ID                    0.000000
Health_Camp_ID                0.000000
Registration_Date             0.456075
Var1                          0.000000
Var2                          0.000000
Var3                          0.000000
Var4                          0.000000
Var5                          0.000000
Online_Follower               0.000000
LinkedIn_Shared               0.000000
Twitter_Shared                0.000000
Facebook_Shared               0.000000
Income                        0.000000
Education_Score               0.000000
Age                           0.000000
First_Interaction             0.000000
City_Type                    44.155154
Employer_Category            79.613886
Camp_Start_Date               0.000000
Camp_End_Date                 0.000000
Category1                     0.000000
Category2                     0.000000
Category3                     0.000000
Donation                     91.759653
Health_Score                 91.759653
Health Score             

In [1630]:
df1.shape

(22584, 28)

In [1631]:
df1.drop('Donation',1,inplace=True)

In [1632]:
df1['Health_Score'].fillna(dum['Health Score']).isnull().sum()

19094

In [1633]:
df1['Health_Score'] = df1['Health_Score'].fillna(df1['Health Score'])

In [1634]:
df1.drop('Health Score',1,inplace=True)
df1.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Health_Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,507518,6529,02/04/06,0,0,0,0,0,0,0,0,0,None,None,None,07-Jan-05,B,NaN,30-Mar-06,03-Apr-06,Second,A,2,0.065742,NaN,NaN
1,511232,6529,03/03/06,0,0,0,0,0,0,0,0,0,None,None,None,29-Jun-04,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,NaN
2,507817,6529,15/02/06,0,0,0,0,0,0,0,0,0,None,None,None,13-Jul-04,E,NaN,30-Mar-06,03-Apr-06,Second,A,2,0.338359,NaN,NaN
3,500528,6529,27/03/06,0,0,0,0,0,0,0,0,0,None,None,None,27-Mar-06,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,NaN,NaN,NaN
4,503972,6529,19/02/06,1,0,0,0,0,0,0,0,0,1,64,63,05-Dec-02,B,Others,30-Mar-06,03-Apr-06,Second,A,2,0.572376,NaN,NaN


In [1635]:
df1.drop(['Number_of_stall_visited','Last_Stall_Visited_Number',
         'Camp_Start_Date','Camp_End_Date','Registration_Date'],1,inplace=True)

In [1636]:
df1.drop('First_Interaction',1,inplace=True)
df1.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,507518,6529,0,0,0,0,0,0,0,0,0,None,None,None,B,NaN,Second,A,2,0.065742
1,511232,6529,0,0,0,0,0,0,0,0,0,None,None,None,NaN,NaN,Second,A,2,NaN
2,507817,6529,0,0,0,0,0,0,0,0,0,None,None,None,E,NaN,Second,A,2,0.338359
3,500528,6529,0,0,0,0,0,0,0,0,0,None,None,None,NaN,NaN,Second,A,2,NaN
4,503972,6529,1,0,0,0,0,0,0,0,0,1,64,63,B,Others,Second,A,2,0.572376


In [1637]:
df1['Health_Score'].median()

0.525731699

In [1638]:
df1['Health_Score'] = df1['Health_Score'].fillna(df1['Health_Score'].median())

In [1639]:
df1.isnull().sum()

Patient_ID               0
Health_Camp_ID           0
Var1                     0
Var2                     0
Var3                     0
Var4                     0
Var5                     0
Online_Follower          0
LinkedIn_Shared          0
Twitter_Shared           0
Facebook_Shared          0
Income                   0
Education_Score          0
Age                      0
City_Type             9972
Employer_Category    17980
Category1                0
Category2                0
Category3                0
Health_Score             0
dtype: int64

In [1640]:
df1['City_Type'] = df1['City_Type'].fillna(df1['City_Type'].mode()[0])

In [1641]:
df1['Employer_Category'] = df1['Employer_Category'].fillna(df1['Employer_Category'].mode()[0])

In [1642]:
df1.isnull().sum()

Patient_ID           0
Health_Camp_ID       0
Var1                 0
Var2                 0
Var3                 0
Var4                 0
Var5                 0
Online_Follower      0
LinkedIn_Shared      0
Twitter_Shared       0
Facebook_Shared      0
Income               0
Education_Score      0
Age                  0
City_Type            0
Employer_Category    0
Category1            0
Category2            0
Category3            0
Health_Score         0
dtype: int64

In [1643]:
df1['Var1'] = df1['Var1'].astype('O')

In [1644]:
df1['Var2'] = df1['Var2'].astype('O')
df1['Var3'] = df1['Var3'].astype('O')
df1['Var4'] = df1['Var4'].astype('O')
df1['Var5'] = df1['Var5'].astype('O')

In [1645]:
df1.describe()

,Patient_ID,Health_Camp_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Category3,Health_Score
count,22584.000000,22584.000000,22584.000000,22584.000000,22584.000000,22584.000000,22584.000000,22584.000000
mean,507302.632217,6547.595156,0.055703,0.067039,0.053224,0.056633,1.994288,0.528164
std,12395.754129,19.290191,0.229353,0.250094,0.224484,0.231145,0.075363,0.114855
min,485680.000000,6523.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.002075
25%,496627.000000,6534.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.525732
50%,507395.500000,6541.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.525732
75%,517925.750000,6562.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.525732
max,528656.000000,6587.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000


In [1646]:
df1.describe(include='O')

,Var1,Var2,Var3,Var4,Var5,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2
count,22584,22584,22584,22584,22584,22584,22584,22584,22584,22584,22584,22584
unique,75,41,2,5,24,8,158,50,9,14,3,7
top,0,0,0,0,0,None,None,None,B,Technology,First,F
freq,20161,21843,22577,22243,20775,16050,19634,15504,12431,19008,14902,7344


In [1647]:
df1['Age'] = df1['Age'].replace('None',np.nan)

In [1648]:
df1['Education_Score'] = df1['Education_Score'].replace('None',np.nan)
df1['Income'] = df1['Income'].replace('None',np.nan)

In [1649]:
df1.describe(include='O')

,Var1,Var2,Var3,Var4,Var5,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2
count,22584,22584,22584,22584,22584,6534,2950,7080,22584,22584,22584,22584
unique,75,41,2,5,24,7,157,49,9,14,3,7
top,0,0,0,0,0,0,75,41,B,Technology,First,F
freq,20161,21843,22577,22243,20775,2511,128,570,12431,19008,14902,7344


In [1650]:
dum['Age'] = dum['Age'].replace('None',np.nan)
dum['Education_Score'] = dum['Education_Score'].replace('None',np.nan)
dum['Income'] = dum['Income'].replace('None',np.nan)

In [1651]:
df1['Age'] = df1['Age'].fillna(df1['Age'].median())
df1['Income'] = df1['Income'].fillna(df1['Income'].median())
df1['Education_Score'] = df1['Education_Score'].fillna(df1['Education_Score'].mode()[0])

In [1652]:
df1.head()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,507518,6529,0,0,0,0,0,0,0,0,0,1.0,75,44.0,B,Technology,Second,A,2,0.065742
1,511232,6529,0,0,0,0,0,0,0,0,0,1.0,75,44.0,B,Technology,Second,A,2,0.525732
2,507817,6529,0,0,0,0,0,0,0,0,0,1.0,75,44.0,E,Technology,Second,A,2,0.338359
3,500528,6529,0,0,0,0,0,0,0,0,0,1.0,75,44.0,B,Technology,Second,A,2,0.525732
4,503972,6529,1,0,0,0,0,0,0,0,0,1,64,63,B,Others,Second,A,2,0.572376


In [1653]:
df1['City_Type'] = LE1.transform(df1['City_Type'])

In [1654]:
df1['Employer_Category'] = LE2.transform(df1['Employer_Category'])

In [1655]:
df1['Category1'] = LE3.transform(df1['Category1'])

In [1656]:
df1['Category2'] = LE4.transform(df1['Category2'])

In [1657]:
df1.drop(['Patient_ID','Health_Camp_ID'],1,inplace=True)

In [1658]:
df1.dtypes

Var1                  object
Var2                  object
Var3                  object
Var4                  object
Var5                  object
Online_Follower        int64
LinkedIn_Shared        int64
Twitter_Shared         int64
Facebook_Shared        int64
Income                object
Education_Score       object
Age                   object
City_Type              int32
Employer_Category      int32
Category1              int32
Category2              int32
Category3              int64
Health_Score         float64
dtype: object

In [1659]:
df1[['Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared']] = df1[['Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared']].astype('O')

In [1660]:
df1.dtypes

Var1                  object
Var2                  object
Var3                  object
Var4                  object
Var5                  object
Online_Follower       object
LinkedIn_Shared       object
Twitter_Shared        object
Facebook_Shared       object
Income                object
Education_Score       object
Age                   object
City_Type              int32
Employer_Category      int32
Category1              int32
Category2              int32
Category3              int64
Health_Score         float64
dtype: object

In [1661]:
df1[['Income','Education_Score','Age']] = df1[['Income','Education_Score','Age']].astype(float)

In [1662]:
df1.head()

,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Health_Score
0,0,0,0,0,0,0,0,0,0,1.0,75.0,44.0,1,11,1,0,2,0.065742
1,0,0,0,0,0,0,0,0,0,1.0,75.0,44.0,1,11,1,0,2,0.525732
2,0,0,0,0,0,0,0,0,0,1.0,75.0,44.0,4,11,1,0,2,0.338359
3,0,0,0,0,0,0,0,0,0,1.0,75.0,44.0,1,11,1,0,2,0.525732
4,1,0,0,0,0,0,0,0,0,1.0,64.0,63.0,1,7,1,0,2,0.572376


In [1663]:
df1.iloc[:,[9,10,11,17]] = SS.transform(df1.iloc[:,[9,10,11,17]])

In [1664]:
ypred1 = model.predict(df1)

In [1665]:
ypred1.shape

(22584,)

In [1667]:
yp = pd.DataFrame(ypred)
yp.value_counts()

0.0    25407
1.0    11478
dtype: int64

In [1668]:
yp1 = pd.DataFrame(ypred1)
yp1.value_counts()

1.0    22562
0.0       22
dtype: int64

In [1204]:
yp1.replace([1,0],[0,1],inplace=True)

In [1205]:
sub = pd.read_csv('Sample_Submission.csv')
sub.head()

,Patient_ID,outcome
0,507518,0
1,516180,0
2,507204,0
3,505815,0
4,501809,0


In [1206]:
sub['outcome'] = yp1

In [1207]:
sub.head()

,Patient_ID,outcome
0,507518,0.0
1,516180,0.0
2,507204,0.0
3,505815,0.0
4,501809,0.0


In [1208]:
sub.to_csv('sub1.csv',index=False)